In [ ]:
#install.packages("stringr")
#install.packages("tidyr")
#install.packages("dplyr")

In [1]:
library(readxl)
library(stringr)
#library(tidyr)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [118]:
MY_STOCK <- list.files("config/stock_matrix/", pattern="\\.xlsx$")
stock <- as.data.frame(read_excel(paste0("config/stock_matrix/", MY_STOCK)))
names(stock)[1] <- "X__1"
rownames(stock) <- stock$X__1
stock <- stock[, -1]
stock <- as.matrix(stock)

New names:
* `` -> ...1



In [119]:
stock
base::rownames(stock)
#base::colnames(stock)[2] <- "xanadu"
stock

,Col-01,Col-02,Col-03,Col-04
Row-01,D-0078,D-0079,D-0080,D-0081
Row-02,D-0102,D-0103,D-0104,D-0105
Row-03,D-0126,D-0127,D-0128,D-0129
Row-04,D-0150,D-0151,D-0152,D-0153


[1] "Row-01" "Row-02" "Row-03" "Row-04"

,Col-01,Col-02,Col-03,Col-04
Row-01,D-0078,D-0079,D-0080,D-0081
Row-02,D-0102,D-0103,D-0104,D-0105
Row-03,D-0126,D-0127,D-0128,D-0129
Row-04,D-0150,D-0151,D-0152,D-0153


In [112]:
# infer SE vs. PE
str(
all_col_samples
    )

 Named chr [1:4] "Col-01_1.fq.gz" "xanadu_1.fq.gz" "Col-03_1.fq.gz" ...
 - attr(*, "names")= chr [1:4] "Col-01" "xanadu" "Col-03" "Col-04"


In [120]:
# compute grid dimension length of row and column
row_dim_length <- length(rownames(stock))
col_dim_length <- length(colnames(stock))

# function returning all files sharing pattern
list_all_files <- function(x){
  list.files(path="rawreads/", pattern=x)  
}

# apply "list_all_files" function to all row & column samples
all_row_samples <- sapply(X = base::rownames(stock), FUN = list_all_files)
all_col_samples <- sapply(X = base::colnames(stock), FUN = list_all_files)

In [121]:
all_row_samples
all_col_samples

Row-01,Row-02,Row-03,Row-04
Row-01_1.fq.gz,Row-02_1.fq.gz,Row-03_1.fq.gz,Row-04_1.fq.gz
Row-01_2.fq.gz,Row-02_2.fq.gz,Row-03_2.fq.gz,Row-04_2.fq.gz


Col-01,Col-02,Col-03,Col-04
Col-01_1.fq.gz,Col-02_1.fq.gz,Col-03_1.fq.gz,Col-04_1.fq.gz
Col-01_2.fq.gz,Col-02_2.fq.gz,Col-03_2.fq.gz,Col-04_2.fq.gz


In [122]:
# decider condition in which mode to continue
# first we check if row/column sample files are themselves consistently matrixes -> thus PE
# in else if, we check if we truly have one sample per row or column position -> thereby SE
# for both SE or PE we write a read_type.yaml containing info of SE/PE - that we can parse later
if (is.matrix(all_row_samples) & is.matrix(all_col_samples)) {
    print("Paired-end data detected - continuing in PE mode")
    PE <- TRUE
    read_type_file <- file("config/read_type.yaml")
    writeLines(c("read_type: PE"), read_type_file)
    close(read_type_file)
} else if (length(all_row_samples) == row_dim_length & length(all_col_samples) == col_dim_length) {
    print("Single-end data detected - continuing in SE")
    PE <- FALSE
    read_type_file <- file("config/read_type.yaml")
    writeLines(c("read_type: SE"), read_type_file)
    close(read_type_file)
} else {
    print("Input data seems incorrect - inconsistent # of fastq files for either SE or PE input set")
}

[1] "Paired-end data detected - continuing in PE mode"


In [123]:
# whether PE or SE (so PE == FALSE) - create appropiate sample and mode sheet
if (PE == TRUE) {
  row_tibble <- tibble(
    dim="row",
    num=seq.int(ncol(all_row_samples)),
    base_name = colnames(all_row_samples),
    fq_1_file = all_row_samples[1,],
    fq_2_file = all_row_samples[2,],
    row.names = NULL  
  )

  col_tibble <- tibble(
    dim="col",
    num=seq.int(ncol(all_col_samples)),
    base_name = colnames(all_col_samples),
    fq_1_file = all_col_samples[1,],
    fq_2_file = all_col_samples[2,],
    row.names = NULL  
  )

  row_col_tibble <- rbind(row_tibble, col_tibble)
  row_col_tibble

  row_col_tibble_final <- row_col_tibble %>%
    mutate(fq_1_end = str_remove(fq_1_file, pattern = c(base::rownames(stock), base::colnames(stock)))) %>%
    mutate(fq_2_end = str_remove(fq_2_file, pattern = c(base::rownames(stock), base::colnames(stock))))
    
  # write generated sample sheet to 'config/grid_sample_sheet.tsv'
  write.table(row_col_tibble_final, 
              file='config/grid_sample_sheet.tsv', 
              quote=FALSE, 
              sep='\t', 
              col.names = TRUE,
              row.names = FALSE)
    
} else if (PE == FALSE) {
    row_tibble <- tibble(
      dim = "row",
      num = seq.int(length(all_row_samples)),
      base_name = names(all_row_samples),
      fq_1_file = all_row_samples
    )

  col_tibble <- tibble(
    dim="col",
    num=seq.int(length(all_col_samples)),
    base_name = names(all_col_samples),
    fq_1_file = all_col_samples
  )

  row_col_tibble <- rbind(row_tibble, col_tibble)

  row_col_tibble_final <- row_col_tibble %>%
    mutate(fq_1_end = str_remove(fq_1_file, pattern = c(base::rownames(stock), base::colnames(stock))))  
    
  # write generated sample sheet to 'config/grid_sample_sheet.tsv'
  write.table(row_col_tibble_final, 
              file='config/grid_sample_sheet.tsv', 
              quote=FALSE, 
              sep='\t', 
              col.names = TRUE,
              row.names = FALSE)
    
} else {
    print("Sth. somewhere went horribly wrong. You should not be able to be here.")
    print("Edge case encountered - contact maintainer of MuWU.")
}


In [109]:
#read_type_file <- file("config/read_type.yaml")
#writeLines(c("read_type: SE"), read_type_file)
#close(read_type_file)


In [108]:
# write generated sample sheet to 'config/grid_sample_sheet.tsv'
#write.table(row_col_tibble_final, 
#            file='config/grid_sample_sheet.tsv', 
#            quote=FALSE, 
#            sep='\t', 
#            col.names = TRUE,
#            row.names = FALSE)